## Dependencies and Setup

Load the target image, and a pair of simulations to compare

In [ ]:
using Revise
using Paint
using Serialization
using Images, ImageShow
using Plots
using Evolutionary
using BlackBoxOptim
using StaticArrays
using ImageFeatures
using IntervalSets
using ReinforcementLearning
using ReinforcementLearningBase
using ReinforcementLearningZoo
using Combinatorics
using StatsBase
using Random

In [ ]:
makecol(x) = RGBA{Float32}(x.r, x.g, x.b, 0.5)

In [ ]:
img = ones(RGB{Float32}, (200, 200))
tri1 = Triangle(Point(0.2, 0.4), Point(0.7, 0.2), Point(0.9, 0.9))
draw!(img, tri1, RGB{Float32}(1, 0, 0), RasterAlgorithmScanline())
img

In [ ]:
function disp(tri)
    newimg = copy(img)
    draw!(newimg, tri, RGBA{Float32}(0, 0, 1, 0.5), RasterAlgorithmScanline())
    newimg
end

In [ ]:
blank = ones(RGB{Float32}, (200, 200))
base = imloss(blank, img, SELoss())
function err(tri)
    col = averagepixel(img, blank, 1.0, tri, RasterAlgorithmScanline())
    base + drawloss(img, blank, tri, col, SELoss(), RasterAlgorithmScanline())
end

In [ ]:
optimtribase = Triangle(Point(0.4, 0.4), Point(0.5, 0.6), Point(0.6, 0.4))
optimtri = optimtribase
println(err(optimtri))
disp(optimtri)

In [ ]:
function refine_allparam_fixed(base, target, basetri ; niters = 1000, factor = 0.005f0)
    for i = 1:niters
        deltas = randn(Float32, 6) .* factor
        newt = mutate(basetri, deltas)
        if err(newt) < err(basetri)
            basetri = newt
        end
    end
    basetri, err(basetri)
end

function refine_allparam_scheduled(base, target, basetri ; niters = 1000, sched=Pair(0.05,0.005))
    for i = 1:niters
        factor = range(sched.first, sched.second, length=niters)[i]
        deltas = randn(Float32, 6) .* factor
        newt = mutate(basetri, deltas)
        if err(newt) < err(basetri)
            basetri = newt
        end
    end
    basetri, err(basetri)
end

function refine_allparam_custom(base, target, basetri ; niters = 1000)
    for i = 1:niters
        factor = 0.001f0
        if i < 250
            factor = range(0.05, 0.005f0, length=250)[i]
        end
        deltas = randn(Float32, 6) .* factor
        newt = mutate(basetri, deltas)
        if err(newt) < err(basetri)
            basetri = newt
        end
    end
    basetri, err(basetri)
end

function refine_oneparam_fixed(base, target, basetri ; niters = 1000, factor = 0.005f0)
    for i = 1:niters
        which = rand(1:6)
        deltas = zeros(6)
        deltas[which] = randn(Float32) * factor
        newt = mutate(basetri, deltas)
        if err(newt) < err(basetri)
            basetri = newt
        end
    end
    basetri, err(basetri)
end

function refine_oneparam_scheduled(base, target, basetri ; niters = 1000, sched=Pair(0.05,0.005))
    for i = 1:niters
        which = rand(1:6)
        deltas = zeros(6)
        factor = range(sched.first, sched.second, length=niters)[i]
        deltas[which] = randn(Float32) * factor
        newt = mutate(basetri, deltas)
        if err(newt) < err(basetri)
            basetri = newt
        end
    end
    basetri, err(basetri)
end

In [ ]:
plotly()

xs = collect(10:100:1000)
arrays = Vector{Vector{Float64}}([])

# for alg in [refine_allparam_fixed, refine_allparam_scheduled, refine_oneparam_fixed, refine_oneparam_scheduled]
for alg in [refine_allparam_scheduled, refine_allparam_custom]
    losses = []
    
    for i in xs
        trials = []
        for k = 1:10
            push!(trials, alg(base, img, optimtribase; niters=i)[2])
        end
        push!(losses, mean(trials))
    end
    push!(arrays, losses)
end

plot(xs, arrays)

In [ ]:
plotly()

xs = collect(10:100:2000)
arrays = Vector{Vector{Float64}}([])

for sched in [Pair(0.01, 0.001), Pair(0.05,0.001), Pair(0.05,0.005), Pair(0.5,0.001), Pair(0.5,0.005), Pair(0.005, 0.0005), Pair(0.01, 0.0005)]
    losses = []
    
    for i in xs
        trials = []
        for k = 1:10
            push!(trials, refine_allparam_scheduled(base, img, optimtribase; niters=i, sched=sched)[2])
        end
        push!(losses, mean(trials))
    end
    push!(arrays, losses)
end

plot(xs, arrays)

In [ ]:
@time tri, loss = refine_allparam_fixed(base, img, optimtribase, niters=1000)
println(loss)
disp(tri)